In [1]:
import ratterdam_CoreDataStructures as Core
import ratterdam_ParseBehavior as Parse
import ratterdam_Defaults as Def
import ratterdam_visBasic as Vis
import utility_fx as util

import numpy as np
from scipy.stats import sem
import os, sys
import matplotlib.gridspec as gridspec
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d

In [2]:
%qtconsole --style native

In [4]:
datafile = "C:\\Users\\whock\\Google Drive\\KnierimLab\\Ratterdam\\Data\\R781\\Beltway_D3\\"
expCode = "BRD3"
alleyTracking, alleyVisits,  txtVisits, p_sess, ts_sess = Parse.getDaysBehavioralData(datafile, expCode)

In [5]:
clustList = util.getClustList(datafile)
unit = Core.UnitData("TT6\\cl-maze1.4", datafile, expCode, Def.alleyBounds, alleyVisits, txtVisits, p_sess, ts_sess)
unit.loadData_raw()

C:\Users\whock\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:138: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*33
C:\Users\whock\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:138: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*33
C:\Users\whock\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:144: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 33
C:\Users\whock\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:144: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 33
C:\Users\whock\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:310: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW
C:\Users\whock\Google Drive\Python_Code\KLab\mts_analysis\utility_fx.py:306: RuntimeWarning: invalid value encountered in multiply
  W=0

## Raster Plots


In [27]:
alley = 7
visits = {txt:[] for txt in ['A','B','C']}
visitIdx = {txt:[] for txt in ['A','B','C']}

maxtime = 0
for i,visit in enumerate(unit.alleys[alley]):
    
    # get this to plot a reasonable divider btwn txts

    if visit['spikes'].shape[0] > 0:
        if max(visit['spikes'][:,0]) > maxtime:
            maxtime = max(visit['spikes'][:,0])
            
    visits[visit['metadata']['stimulus']].append(visit['spikes'])
    visitIdx[visit['metadata']['stimulus']].append(i)

In [30]:
# Raster of spikes arranged by ts

rsum = 0
for txt in ['A','B','C']:
    for i in range(len(visits[txt])):
        if visits[txt][i].shape[0] >0:
            plt.scatter(visits[txt][i][:,0]-visits[txt][i][0,0], [rsum]*visits[txt][i].shape[0], s=12, c='k',alpha=0.5)
        rsum += 1
    plt.hlines(rsum,0,5e7,color='k',linestyle='--')

In [70]:
# Spatial raster of spikes arranged by linear position on alley
# ul, ll, ur, lr = util.extractCorners(Def.alleyBounds[alley-1])
numP = 1000
mpL = midpoint(ll,ul)
mpR = midpoint(lr,ur)
interpFx = interp1d([mpL[0], mpR[0]], [mpL[1], mpR[1]])
segmentX = np.linspace(mpL[0], mpR[0], numP)
segmentY = interpFx(segmentX)
segment = list(zip(segmentX, segmentY))

projP = {txt:[] for txt in ['A', 'B', 'C']}
for txt in ['A', 'B', 'C']:
    for i in range(len(visits[txt])):
        if visits[txt][i].shape[0] > 0:
            projP[txt].append(util.project_points(segment, visits[txt][i][:,1:]))
        else:
            projP[txt].append([])

In [72]:
fig, ax = plt.subplots()
rsum = 0
labelidx = []
for txt in ['A','B','C']:
    labelidx.append(rsum)
    for i in range(len(projP[txt])):
        if type(projP[txt][i]) == np.ndarray and projP[txt][i].shape[0] >0:
            fig.axes[0].scatter(projP[txt][i], [rsum]*projP[txt][i].shape[0], s=12, c='k',alpha=0.5)
        rsum += 1
    fig.axes[0].hlines(rsum,0,numP,color='k',linestyle='--')
fig.axes[0].set_yticks(labelidx)
fig.axes[0].set_yticklabels(['A','B','C'])

[Text(0,0,'A'), Text(0,0,'B'), Text(0,0,'C')]

## Fixing Behavioral Data loading for R808
##### And in so doing alter pipeline so behavioral data, if not also neural data structs, are saved and not loaded raw each time. make it a bit more manual in the beginning then stable after